# 0. 설정

In [2]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/MyDrive/colab

/content/drive/MyDrive/colab


In [15]:
pwd

'/content/drive/My Drive/colab'

# 1. 텐서

In [2]:
import torch

In [3]:
# 벡터 정의
vector1 = torch.tensor([1., 2., 3.])
vector2 = torch.tensor([4., 5., 6.])

In [4]:
# torch 내장함수를 이용한 벡터wise 연산
torch.add(vector1, vector2) # 같은 위치끼리 더하고
torch.sub(vector1, vector2) # 빼고
torch.mul(vector1, vector2) # 곱하고
torch.div(vector1, vector2) # 나누고

tensor([0.2500, 0.4000, 0.5000])

In [5]:
# 행렬 정의
matrix1 = torch.tensor([[1.,2.], [3.,4.]])
matrix2 = torch.tensor([[5.,6.], [7.,8.]])

In [6]:
torch.add(matrix1 , matrix2) # 같은 위치끼리 더하고
torch.sub(matrix1 , matrix2) # 빼고
torch.mul(matrix1 , matrix2) # 곱하고
torch.div(matrix1 , matrix2) # 나누고
torch.matmul(matrix1, matrix2) # 행렬곱

tensor([[19., 22.],
        [43., 50.]])

In [7]:
# 텐서 정의
tensor1 = torch.tensor([ [[1.,2.], [3.,4.]],
                         [[5.,6.], [7.,8.]] ])
print(tensor1)

tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])


In [8]:
# 텐서 연산
torch.matmul(tensor1, tensor1)

tensor([[[  7.,  10.],
         [ 15.,  22.]],

        [[ 67.,  78.],
         [ 91., 106.]]])

# 2. Autograd

Back propagation(역전파)를 이용해서 파라미터를 업데이트할 때 Autograd 방식으로 쉽게 구현할 수 있음. 

In [9]:
if torch.cuda.is_available():
  DEVICE = torch.device("cuda")
else:
  DEVICE = torch.device("cpu")

`batch_size` : 딥러닝 모델에서 파라미터를 업데이트할 때 계산되는 데이터의 개수  
`input_size` : 입력층의 노드 수  
- `input_size`의 크기를 가지는 데이터를 `batch_size`만큼 사용  (아래 예제에서는 (64, 1000))  
`hidden_size` : 은닉층의 노드 수   
`output_size` : 출력되는 벡터 크기  

In [10]:
batch_size = 64 
input_size = 1000
hidden_size = 100
output_size = 10

In [11]:
# sample data 생성
# 입력층
x = torch.randn(batch_size,
                input_size,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = False)

# 은닉층1, 역전파를 통해 업데이트 해야 하는 대상
w1 = torch.randn(input_size,
                hidden_size,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = True)

# 은닉층2, 역전파를 통해 업데이트 해야 하는 대상
w2 = torch.randn(hidden_size,
                output_size,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = True)

# 출력층
y = torch.randn(batch_size,
                output_size,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = False)

In [18]:
x.size()

torch.Size([64, 10])

In [19]:
w1.size()

torch.Size([1000, 100])

In [20]:
lr = 1e-6 # learning rate
for t in range(1, 501): # epoch = 500
  y_pred = x.mm(w1).clamp(min = 0).mm(w2) # 예측값 = x와 w1 행렬곱 -> clamp(min=0 이므로, ReLU와 같은 활성화 함수)적용 -> w2 행렬곱 
  
  loss = (y_pred-y).pow(2).sum() # SSE
  if t % 100 == 0:
    print(f"Iteration: {t} \t Loss: {loss.item()}")
  loss.backward() # 각 파라미터 값에 대해 gradient를 계산 후 역전파 진행

  with torch.no_grad(): 
    # gradient 업데이트 
    w1 -= lr * w1.grad 
    w2 -= lr * w2.grad
    
    # gradient 초기화
    w1.grad.zero_()
    w2.grad.zero_()

Iteration: 100 	 Loss: 1038.6949462890625
Iteration: 200 	 Loss: 8.037734985351562
Iteration: 300 	 Loss: 0.10370723903179169
Iteration: 400 	 Loss: 0.0019828418735414743
Iteration: 500 	 Loss: 0.00016050023259595037


`loss.backward()` 이전까지  
- 2개의 hidden layer를 한번 거쳐 손실함수를 계산하는 과정  

`loss.backward()`  
- `requires_grad = True`인 텐서에 대해 그래디언트값을 계산 ($ dLoss/dw $) 
- `w1.grad`, `w2.grad`로 호출 가능해짐  

`with torch.no_grad():`   
- 텐서를 정의할 때 gradient 연산 옵션을 끌 때, requires_grad = False로 만들어줌
- 메모리 사용량을 아낌
- 각 파라미터 값에 대해 그래디언트를 계산한 결과를 이용해 파라미터 값을 업데이트 할때, 해당 시점의 그래디언트 값을 고정한다는 의미


## clamp 함수 설명
위 예시에서는 min = 0 이고, max는 없기 때문에, ReLU 함수와 똑같다.

\begin{cases}
min,\;if\;x<min\\
x,\;if\;min\leq x\leq\max\\
max, \; if\; x > max
\end{cases}

# 추가로 공부해볼 것들
1. Back Propagation (역전파)
2. Gradient Descent Method